In [1]:
from model import load_model
from pyspark.sql import SparkSession
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from tqdm.auto import tqdm
import findspark
import pandas as pd
import spacy
import torch
import torch.nn as nn

tqdm.pandas()
encoder = LabelEncoder()

nlp = spacy.load('en_core_web_lg')

2022-12-07 20:41:51.490975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 20:41:53.353566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:41:53.353634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:41:53.353641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
findspark.init()
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('word2vec') \
    .config('spark.driver.maxResultSize', '8g') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/07 20:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Good to go!


In [4]:
df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')

In [5]:
selected = pd.DataFrame(df.take(640), columns=['text'])
selected['vector'] = selected['text'].progress_apply(lambda x: nlp(x).vector)

In [10]:
def collate_batch(batch):
    embedding_list = [torch.tensor(_embedding, dtype=torch.float32) for _embedding in batch]
    embedding_list = torch.stack(embedding_list)

    return embedding_list.to(device)

In [18]:
model = load_model('/common/users/shared/cs543_fall22_group3/models/class_model.pt')
model

TextClassificationModel(
  (conv): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=18944, out_features=42, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [15]:
def predict(dataloader):
    model.eval()
    pred = []

    with torch.no_grad():
        for idx, vector in enumerate(dataloader):
            predicted_label = model(vector)
            pred.extend(predicted_label.argmax(1).cpu().detach().numpy())

    return pred

In [16]:
classify_dataset = to_map_style_dataset(selected['vector'])
test_dataloader = DataLoader(classify_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [19]:
pred = predict(test_dataloader)
selected['predictions'] = pred

In [24]:
category_dict = {
    0: 'ARTS',
    1: 'ARTS & CULTURE',
    2: 'BLACK VOICES',
    3: 'BUSINESS',
    4: 'COLLEGE',
    5: 'COMEDY',
    6: 'CRIME',
    7: 'CULTURE & ARTS',
    8: 'DIVORCE',
    9: 'EDUCATION',
    10: 'ENTERTAINMENT',
    11: 'ENVIRONMENT',
    12: 'FIFTY',
    13: 'FOOD & DRINK',
    14: 'GOOD NEWS',
    15: 'GREEN',
    16: 'HEALTHY LIVING',
    17: 'HOME & LIVING',
    18: 'IMPACT',
    19: 'LATINO VOICES',
    20: 'MEDIA',
    21: 'MONEY',
    22: 'PARENTING',
    23: 'PARENTS',
    24: 'POLITICS',
    25: 'QUEER VOICES',
    26: 'RELIGION',
    27: 'SCIENCE',
    28: 'SPORTS',
    29: 'STYLE',
    30: 'STYLE & BEAUTY',
    31: 'TASTE',
    32: 'TECH',
    33: 'THE WORLDPOST',
    34: 'TRAVEL',
    35: 'U.S. NEWS',
    36: 'WEDDINGS',
    37: 'WEIRD NEWS',
    38: 'WELLNESS',
    39: 'WOMEN',
    40: 'WORLD NEWS',
    41: 'WORLDPOST'
}

In [26]:
selected['prediction_label'] = selected['predictions'].apply(lambda x: category_dict[x])
selected.head()